In [1]:
from ngsolve import *
from ngsolve.webgui import Draw
from netgen.occ import *
from coil_geometry import create_coil_geometry
import matplotlib.pyplot as plt
import numpy as np 

In [2]:
nwindings = 6  # Number of windings
wireradius = 0.001  # Radius of the wire
coilradius = 0.01  # Radius of the coil

# Geometrie laden
geo, crosssection = create_coil_geometry(nwindings, wireradius, coilradius)

In [ ]:
mesh = Mesh(
    geo.GenerateMesh(grading=0.2)
)
mesh.Curve(1) 

In [ ]:
print("Materialien im Mesh:", mesh.GetMaterials())
print("Randbedingungen im Mesh:", mesh.GetBoundaries())

In [5]:
# Konstanten
mu = 4 * pi * 1e-7      # Magnetische Feldkonstante (S.2)
sigma_coil = 5.87e7     # Leitfähigkeit Kupfer (S.11)
sigma_air = 0           # Leitfähigkeit Luft
k = 0.1                 # Regularisierungfaktor
i_c = 100               # Strom [A]

# Geometrische Parameter
A_c = pi * wireradius**2  
crosssection = A_c
r = sqrt(x**2 + y**2)

# Get j_c from aufgabe 3 

In [6]:
# H(curl)-Raum definieren
order = 4
V = HCurl(mesh, order=order-1, nograds=True, dirichlet="outer")
u, v = V.TnT()
gfA = GridFunction(V)

V_phi = H1(mesh,order = order, dirichlet='out',definedon=mesh.Materials('coil'))
phi, psi = V_phi.TnT()
gfphi = GridFunction(V_phi)


a_phi = BilinearForm(V_phi)
a_phi += sigma_coil * grad(phi) * grad(psi) * dx

# Rechte Seite (Randbedingungen)
f_phi = LinearForm(V_phi)
f_phi += i_c / crosssection * psi *ds("in")

# pre = Preconditioner(a_phi, "multigrid")

with TaskManager():
    a_phi.Assemble()
    f_phi.Assemble()
    
# inv = krylovspace.CGSolver(a_phi.mat, pre.mat, tol=1e-12, printrates=True)

with TaskManager():
    gfphi.vec.data = a_phi.mat.Inverse(V_phi.FreeDofs()) * f_phi.vec
    

In [7]:
# Elektrisches Feld und Stromdichte aus φ berechnen
E = -grad(gfphi)  # Elektrisches Feld
j_c = sigma_coil * E  # Stromdichte

In [ ]:
# Bilinearform und Linearform
a = BilinearForm(V, symmetric=True)
a += (1/mu) * InnerProduct(curl(u), curl(v)) * dx
a += k * InnerProduct(u, v) * dx

# Linearform: Stromdichte
f = LinearForm(V)
f += InnerProduct(j_c, v) * dx("coil")

# Preconditioner
pre = Preconditioner(a, "multigrid")

# Assemble
with TaskManager():
    a.Assemble()
    f.Assemble()

In [ ]:
# Grösse von Gleichungsystems Elemente überprüfen
print(f"Matrix size: {a.mat.height} x {a.mat.width}")
print(f"Vector size: {f.vec.size}")
print(f"Präkonditionierer-Matrixgröße: {pre.mat.height} x {pre.mat.width}")

In [ ]:
# Lösen des Gleichungssystems mit CG-Verfahren und Präkonditionierer (krylovspace.CGSolver)
inv = krylovspace.CGSolver(a.mat, pre.mat, tol=1e-12, printrates = True)

with TaskManager():
    gfA.vec.data = inv * f.vec

In [ ]:
# Calculate the residual manually
residual_vec = f.vec - a.mat * gfA.vec
residual_norm = residual_vec.Norm()
print(f"Residual norm: {residual_norm}")

In [12]:
B = curl(gfA) 

In [ ]:
# Feldlinien to be done 
yiB = np.linspace(-.005,.005,9)
XiB, YiB = np.meshgrid(yiB,yiB)
fieldlines = B._BuildFieldLines(mesh, np.array([XiB.flatten(),
    YiB.flatten(),np.zeros_like(XiB.flatten())]).T,
    num_fieldlines=10**3//5, randomized=False,
    length=1.,thickness=8e-4)
Draw (B, mesh.Materials('coil'), objects=[fieldlines],
      draw_surf=False,
      min=0, max=0.1,
      settings={"Objects": {"Edges": False, "Surface": True,
                            "Wireframe": False}})

In [ ]:
# Anzahl der Punkte im Gitter
nx = 71
nz = 71
lim = 0.02
xvals = np.linspace(-lim, lim, nx)
zvals = np.linspace(-lim, lim, nz)

# Meshgrid anlegen, für (X,Z)
X, Z = np.meshgrid(xvals, zvals)

# Arrays für Bx (U), Bz (V) und Magnetfeldstärke (Mag) anlegen
U = np.zeros_like(X)
V = np.zeros_like(X)
Mag = np.zeros_like(X)  # Betrag des Magnetfelds
Bz = np.zeros_like(X)  # Für Bz-Komponente

# Schleife über Gitterpunkte und Magnetfeld auswerten
for i in range(nz):
    for j in range(nx):
        x = X[i, j]
        z = Z[i, j]

        B_val = B(mesh(x, 0, z))  # B_val enthält (Bx, By, Bz)
        U[i, j] = B_val[0]       # x-Komponente
        V[i, j] = B_val[2]       # z-Komponente
        Bz[i, j] = B_val[2]      # z-Komponente
        Mag[i, j] = np.sqrt(B_val[0]**2 + B_val[2]**2)  # Betrag des Magnetfelds

# Schrittweite für Pfeildichte
step = 6
X_sparse = X[::step, ::step]
Z_sparse = Z[::step, ::step]
U_sparse = U[::step, ::step]
V_sparse = V[::step, ::step]
Mag_sparse = Mag[::step, ::step]

# Plot erstellen
plt.figure(figsize=(10, 7))
plt.title("Magnetfeld in der Ebene y=0")

# Farbkontur basierend auf Magnetfeldbetrag
cont = plt.contourf(X, Z, Bz, levels=50, cmap="viridis", alpha=0.9, antialiased=True)
plt.colorbar(cont, label="$|\\mathbf{B_z}|$ [Tesla]")

# Pfeile für Magnetfeldrichtung und -stärke
plt.quiver(X_sparse, Z_sparse, U_sparse, V_sparse, color='k', scale=4, width=0.003, label='arrow')

# Magnetfeldlinien 
plt.streamplot(X, Z, U, V, color='b', density=1.3, linewidth=0.8)

# Layout
plt.axis('equal')
plt.xlabel("x [m]")
plt.ylabel("z [m]")

# Fix for legend
plt.legend(["Stärke Magnetfeld", "Magnetfeldlinien"], loc='upper right')

plt.show()

# Exports

In [ ]:
import os
project_directory = os.getcwd()
i_c_str = str(i_c)

target_folder_name = "results/inhomo/" + i_c_str  # Name of the folder to check/create

target_folder_path = os.path.join(project_directory, target_folder_name)

# Check if the folder exists
if not os.path.exists(target_folder_path):
    # Create the folder if it doesn't exist
    os.makedirs(target_folder_path)
    os.makedirs(target_folder_path + "/results")
    os.makedirs(target_folder_path + "/plot")
    print(f"Folder '{target_folder_name}' created.")
else:
    # Skip creation if it exists
    print(f"Folder '{target_folder_name}' already exists, skipping creation.")

In [ ]:
override = 1
ord_str = str(order)
ampere_dir = "results/inhomo/" + i_c_str
filename_gfA    = ampere_dir + "/results/gfA_inhomo_" + i_c_str + "_" + ord_str + "ord.vec"
filename_mesh   = ampere_dir + "/results/mesh_inhomo_" + i_c_str + "_" + ord_str + "ord.vol"

if override == 1:
    gfA.Save(filename_gfA)
    
    ngmesh = mesh.ngmesh
    ngmesh.Save(filename_mesh)
    
    print(" ########################### ")
    print(" ------ Results saved ------ ")
    print(" ########################### ")
else:
    print(" ########################### ")
    print(" ---- Results not saved ---- ")
    print(" ########################### ")

In [ ]:
override = 1

filename_magnetfeld_plot   = ampere_dir + "/plot/field_inhomo_plot" + i_c_str + "_" + ord_str + "ord.pdf"

if override == 1:
    # Plot erstellen
    plt.figure(figsize=(10, 7))
    plt.title("Magnetfeld in der Ebene y=0")

    # Farbkontur basierend auf Magnetfeldbetrag
    cont = plt.contourf(X, Z, Bz, levels=50, cmap="viridis", alpha=0.9, antialiased=True)
    plt.colorbar(cont, label="$|\\mathbf{B_z}|$ [Tesla]")

    # Pfeile für Magnetfeldrichtung und -stärke
    plt.quiver(X_sparse, Z_sparse, U_sparse, V_sparse, color='k', scale=4, width=0.003, label='arrow')

    # Magnetfeldlinien 
    plt.streamplot(X, Z, U, V, color='b', density=1.3, linewidth=0.8)

    # Layout
    plt.axis('equal')
    plt.xlabel("x [m]")
    plt.ylabel("z [m]")

    # Fix for legend
    plt.legend(["Stärke Magnetfeld", "Magnetfeldlinien"], loc='upper right')
    plt.savefig(filename_magnetfeld_plot, format='pdf', bbox_inches='tight')
    plt.close()
    
    print(" ########################### ")
    print(" ------ Plot saved ------ ")
    print(" ########################### ")
else:
    print(" ########################### ")
    print(" ---- Plot not saved ---- ")
    print(" ########################### ")